# Restart and resolution change

Fluidsim provides 2 command-line tools to restart a simulation (`fluidsim-restart`) 
and create a new file with a modified resolution (`fluidsim-modif-resolution`). 
Let's present these very useful tools!

This notebook uses a Bash kernel, which means that all the code in the code cells is in Bash!

First, one can get help by invoking these tools with the `-h` option:

In [ ]:
fluidsim-restart -h

In [ ]:
fluidsim-modif-resolution -h

Then, let's use these tools. First we need a simulation directory. 

In [ ]:
export FLUIDSIM_PATH=/tmp/sim_data
mkdir -p $FLUIDSIM_PATH
rm -rf $FLUIDSIM_PATH/doc_aniso

In [ ]:
python -c 'from fluidsim.util.scripts.turb_trandom_anisotropic import main; main()' \
    --sub-directory "doc_aniso" -nz 12 --ratio-nh-nz 2 --t_end 1 \
    --modify-params "params.output.periods_print.print_stdout = 0.25; params.output.periods_save.spect_energy_budg = 0"

We then define a variable with the path of the directory containing the results of the simulation.

In [ ]:
PATH_RUN=$FLUIDSIM_PATH/doc_aniso/$(ls $FLUIDSIM_PATH/doc_aniso)
echo $PATH_RUN

In [ ]:
ls $PATH_RUN

Let's restart the simulation to run until `t_end = 2`:

In [ ]:
fluidsim-restart $PATH_RUN --t_end 2.0

In [ ]:
ls $PATH_RUN

Now, another simulation but with the spatiotemporal spectra output activated:

In [ ]:
fluidsim-restart $PATH_RUN --t_end 3.0 --modify-params "params.output.periods_save.spatiotemporal_spectra = 2 * pi / (4 * params.N)"

In [ ]:
ls $PATH_RUN

In [ ]:
ls $PATH_RUN/spatiotemporal

We want to start a larger simulation from the last state of this small simulation. We first need to create a larger file:

In [ ]:
fluidsim-modif-resolution $PATH_RUN 3/2

In [ ]:
PATH_NEW_STATE=$PATH_RUN/$(ls $PATH_RUN | grep State_phys)
echo $PATH_NEW_STATE

In [ ]:
ls $PATH_NEW_STATE

In [ ]:
fluidsim-restart $PATH_NEW_STATE --t_end 3.1 --modify-params "params.nu_2 /= 2; params.output.periods_print.print_stdout = 0.05"